In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import numpy as np
import sys
#from torch.utils import save_checkpoint, load_checkpoint
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from PIL import ImageFilter
from rembg import remove
from PIL import Image

C:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
import os
from PIL import Image

def preprocess_images(source_folder, target_folder):
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    for file_name in os.listdir(source_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(source_folder, file_name)
            image = Image.open(file_path).resize((512,512))
            processed_image = remove(image, bgcolor=(255, 255, 255,255)).convert('L').convert('RGB')
            target_file_path = os.path.join(target_folder, file_name)
            processed_image.save(target_file_path)
            print(f"Processed and saved: {target_file_path}")

source_folder = 'Humans'
target_folder = 'Pre-Soy'
preprocess_images(source_folder, target_folder)

Processed and saved: Pre-Soy\1 (1).jpeg
Processed and saved: Pre-Soy\1 (1).jpg
Processed and saved: Pre-Soy\1 (1).png
Processed and saved: Pre-Soy\1 (10).jpeg
Processed and saved: Pre-Soy\1 (10).jpg
Processed and saved: Pre-Soy\1 (10).png
Processed and saved: Pre-Soy\1 (100).jpg
Processed and saved: Pre-Soy\1 (100).png
Processed and saved: Pre-Soy\1 (1000).jpg
Processed and saved: Pre-Soy\1 (1001).jpg
Processed and saved: Pre-Soy\1 (1002).jpg
Processed and saved: Pre-Soy\1 (1003).jpg
Processed and saved: Pre-Soy\1 (1004).jpg
Processed and saved: Pre-Soy\1 (1005).jpg
Processed and saved: Pre-Soy\1 (1006).jpg
Processed and saved: Pre-Soy\1 (1007).jpg
Processed and saved: Pre-Soy\1 (1008).jpg
Processed and saved: Pre-Soy\1 (1009).jpg
Processed and saved: Pre-Soy\1 (101).jpg
Processed and saved: Pre-Soy\1 (101).png
Processed and saved: Pre-Soy\1 (1010).jpg
Processed and saved: Pre-Soy\1 (1011).jpg
Processed and saved: Pre-Soy\1 (1012).jpg
Processed and saved: Pre-Soy\1 (1013).jpg
Processed

C:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Выходные данные были обрезаны до нескольких последних строк (5000).
Processed and saved: Pre-Soy\1 (2910).jpg
Processed and saved: Pre-Soy\1 (2911).jpg
Processed and saved: Pre-Soy\1 (2912).jpg
Processed and saved: Pre-Soy\1 (2913).jpg
Processed and saved: Pre-Soy\1 (2914).jpg
Processed and saved: Pre-Soy\1 (2915).jpg
Processed and saved: Pre-Soy\1 (2916).jpg
Processed and saved: Pre-Soy\1 (2917).jpg
Processed and saved: Pre-Soy\1 (2918).jpg
Processed and saved: Pre-Soy\1 (2919).jpg
Processed and saved: Pre-Soy\1 (292).jpg
Processed and saved: Pre-Soy\1 (2920).jpg
Processed and saved: Pre-Soy\1 (2921).jpg
Processed and saved: Pre-Soy\1 (2922).jpg
Processed and saved: Pre-Soy\1 (2923).jpg
Processed and saved: Pre-Soy\1 (2924).jpg
Processed and saved: Pre-Soy\1 (2925).jpg
Processed and saved: Pre-Soy\1 (2926).jpg
Processed and saved: Pre-Soy\1 (2927).jpg
Processed and saved: Pre-Soy\1 (2928).jpg
Processed and saved: Pre-Soy\1 (2929).jpg
Processed and saved: Pre-Soy\1 (293).jpg
Processed 

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

46

In [ ]:
class SoyBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride):

    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 4, padding = 1, bias = True, padding_mode = 'reflect'),
        nn.InstanceNorm2d(out_channels),
        nn.LeakyReLU(0.2)
    )

  def forward(self, x):
     return self.conv(x)


class CumBlock(nn.Module):
  def __init__(self, in_channels, out_channels, down = True, use_act = True, **kwargs):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, padding_mode = 'reflect', **kwargs)
        if down
        else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
        nn.InstanceNorm2d(out_channels),
        nn.ReLU(inplace = True) if use_act else nn.Identity()
    )

  def forward(self, x):
    return self.conv(x)


class RusBlock(nn.Module):
  def __init__(self, channels):
    super().__init__()
    self.block = nn.Sequential(
        CumBlock(channels, channels, kernel_size = 3, padding = 1),
        CumBlock(channels, channels, use_act = False, kernel_size = 3, padding = 1)
    )

  def forward(self, x):
    return x + self.block(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size=4, stride=2, padding=1, padding_mode='reflect'),
            nn.LeakyReLU(0.2)
        )

        layers = []
        in_features = features[0]
        for feature in features[1:]:
            layers.append(
                nn.Sequential(
                    nn.Conv2d(in_features, feature, kernel_size=4, stride=2, padding=1, padding_mode='reflect'),
                    nn.InstanceNorm2d(feature),
                    nn.LeakyReLU(0.2)
                )
            )
            in_features = feature

        layers.append(nn.Conv2d(in_features, 1, kernel_size=4, stride=1, padding=1, padding_mode='reflect'))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        x = self.model(x)
        return torch.sigmoid(x)


In [ ]:
class Generator(nn.Module):
    def __init__(self, img_channels, num_residuals=12):
        super().__init__()
        num_features = 64
        self.initial = nn.Sequential(
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode='reflect'),
            nn.ReLU(inplace=True)
        )
        self.down_blocks = nn.ModuleList(
            [
                CumBlock(num_features, num_features*2, kernel_size=3, stride=2, padding=1),
                CumBlock(num_features*2, num_features*4, kernel_size=3, stride=2, padding=1)
            ]
        )
        self.residual_blocks = nn.Sequential(
            *[RusBlock(num_features*4) for _ in range(num_residuals)]
        )
        self.up_blocks = nn.ModuleList(
            [
                CumBlock(num_features*4, num_features*2, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
                CumBlock(num_features*2, num_features, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
            ]
        )
        self.last = nn.Conv2d(num_features, img_channels, kernel_size=7, stride=1, padding=3, padding_mode='reflect')

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.residual_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, rootA, rootB, transform=None):
        self.transform = transform
        self.files_A = sorted([os.path.join(rootA, file) for file in os.listdir(rootA) if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg')])
        self.files_B = sorted([os.path.join(rootB, file) for file in os.listdir(rootB) if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg')])

    def __getitem__(self, index):
        image_A = remove(Image.open(self.files_A[index % len(self.files_A)])).convert('RGB')
        image_B = Image.open(self.files_B[index % len(self.files_B)]).convert('RGB')
        if self.transform:
            image_A = self.transform(image_A)
            image_B = self.transform(image_B)
        return {"A": image_A, "B": image_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))


transform = transforms.Compose([
    transforms.Resize((128, 128)), #(256, 256)
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageDataset(rootA='Pre-Soy', rootB='Wojak MEGA Collection', transform=transform) # soyjaks   Wojak MEGA Collection
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.cuda.amp import GradScaler, autocast

G_A2B = Generator(img_channels=3)
G_B2A = Generator(img_channels=3)
D_A = Discriminator(in_channels=3)
D_B = Discriminator(in_channels=3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G_A2B.to(device)
G_B2A.to(device)
D_A.to(device)
D_B.to(device)

Discriminator(
  (initial): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
 

In [ ]:
def threshold_images(images, threshold=0.3):
    return (images > threshold).float()


In [ ]:
def train_cycleGAN_with_amp(dataloader, G_A2B, G_B2A, D_A, D_B, epochs=10, device='cuda'):
    scaler = GradScaler()
    optimizer_G = optim.Adam(list(G_A2B.parameters()) + list(G_B2A.parameters()), lr=0.000001, betas=(0.5, 0.999))
    optimizer_D_A = optim.Adam(D_A.parameters(), lr=0.000001, betas=(0.5, 0.999))
    optimizer_D_B = optim.Adam(D_B.parameters(), lr=0.000001, betas=(0.5, 0.999))

    criterion_GAN = nn.MSELoss()
    criterion_cycle = nn.L1Loss()
    criterion_identity = nn.L1Loss()
    for epoch in range(epochs):
        for i, batch in enumerate(dataloader):
            real_A = batch['A'].to(device)
            real_B = batch['B'].to(device)

            valid = torch.ones(real_A.size(0), 1, device=device, requires_grad=False)
            fake = torch.zeros(real_A.size(0), 1, device=device, requires_grad=False)

            optimizer_G.zero_grad()

            with autocast():
                loss_id_A = criterion_identity(G_B2A(real_A), real_A)
                loss_id_B = criterion_identity(G_A2B(real_B), real_B)
                fake_B = G_A2B(real_A)
                thresholded_fake_B = threshold_images(fake_B)
                thresholded_real_B = threshold_images(real_B)
                loss_GAN_A2B = criterion_GAN(D_B(thresholded_fake_B), valid)
                # GAN loss
                #fake_B = G_A2B(real_A)
                #loss_GAN_A2B = criterion_GAN(D_B(fake_B), valid)
                fake_A = G_B2A(real_B)
                loss_GAN_B2A = criterion_GAN(D_A(fake_A), valid)

                recovered_A = G_B2A(fake_B)
                loss_cycle_ABA = criterion_cycle(recovered_A, real_A)
                recovered_B = G_A2B(fake_A)
                loss_cycle_BAB = criterion_cycle(recovered_B, real_B)

                loss_G = loss_id_A + loss_id_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
                #loss_G = loss_id_A + 5*loss_GAN_A2B# + loss_cycle_ABA

            scaler.scale(loss_G).backward()

            scaler.step(optimizer_G)
            scaler.update()
            optimizer_G.zero_grad()

            with autocast():
                optimizer_D_A.zero_grad()
                loss_real = criterion_GAN(D_A(real_A), valid)
                fake_A_ = fake_A.detach()
                loss_fake = criterion_GAN(D_A(fake_A_), fake)
                loss_D_A = (loss_real + loss_fake) / 2

                scaler.scale(loss_D_A).backward()
                scaler.step(optimizer_D_A)
                optimizer_D_A.zero_grad()

                optimizer_D_B.zero_grad()

                loss_real = criterion_GAN(D_B(real_B), valid)
                fake_B_ = fake_B.detach()
                loss_fake = criterion_GAN(D_B(fake_B_), fake)
                loss_D_B = (loss_real + loss_fake) / 2

                scaler.scale(loss_D_B).backward()
                scaler.step(optimizer_D_B)
                optimizer_D_B.zero_grad()

            scaler.update()
            if(i%1 == 0):
              print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] [D loss: {loss_D_A.item() + loss_D_B.item()}] [G loss: {loss_G.item()}]")
              display(convert_face_to_wojak('test_rem.jpg'))
              #display(convert_face_to_wojak('rostisoyjak_rem.jpg').convert('L'))



In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [ ]:
train_cycleGAN_with_amp(dataloader, G_A2B, G_B2A, D_A, D_B)

In [ ]:
from PIL import Image, ImageFilter
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, ToPILImage
from IPython.display import display

transform = Compose([
    Resize((256, 256)),
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
imageee = Image.open('test.jpg')
imageee = remove(imageee, bgcolor=(255, 255, 255,255)).convert('L').convert('RGB')
imageee.save('test_rem.jpg')

imageee = Image.open('rostisoyjak.jpg')
imageee = remove(imageee, bgcolor=(255, 255, 255,255)).convert('L').convert('RGB')
imageee.save('rostisoyjak_rem.jpg')

def convert_face_to_wojak(image_path):
    image = Image.open(image_path).convert('RGB')
    #image = remove(image).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        fake_image = G_A2B(image).detach().cpu()

    img = ToPILImage()(fake_image.squeeze(0))

    return img.resize((512, 512)).convert('L')

G_A2B.eval()
img = convert_face_to_wojak('test.jpg')
img_sharpened_resized = img.filter(ImageFilter.SHARPEN).resize((512, 512)).convert('L')
img_sharpened_resized

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch

In [ ]:
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")

In [ ]:
prompt = 'soyjak with open mouth'
upscaled_image = pipeline(prompt=prompt, image=Image.open('input.png')).images[0]
upscaled_image